In [10]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/Users/apple/Desktop/Kaggle-Competition-Titanic/models')
from collections import Counter
import kmodes

In [11]:
pd.read_csv('train_processed.csv', usecols=range(1,9)).head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,1,1,0,0,2
1,1,1,1,2,1,0,3,0
2,1,3,1,1,0,0,1,2
3,1,1,1,2,1,0,3,2
4,0,3,0,2,0,0,1,2


In [55]:
# kmodes Model with KNN
data = 'train_processed.csv'
X = kmodes.get_data(data)
# print(X[:4])
num_instances = len(X)
# store the best records in 5 iterations 
best_score = 0
best_centroids = []
best_clusters =[]
best_iteratoin = 0
for i in range(10):
    centroids, clusters , iteration= kmodes.find_centers(X, 2)
    purity = kmodes.get_purity(clusters, centroids, num_instances)
    if purity > best_score:
        best_centroids = centroids
        best_clusters = clusters
        best_score = purity
        best_iteratoin = iteration
actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str).values.flatten()
test_data = pd.read_csv('test_processed.csv', usecols=range(1,9),dtype=str).values.tolist()
for idx, c in enumerate(best_centroids):
    labels = np.array(best_clusters[idx])[:,-1]
    label = Counter(labels).most_common(1)[0][0]
    c[-1] = label
count = 0
for i, x in enumerate(test_data):
    distance = [kmodes.get_distance(x, c[:-1]) for c in best_centroids]
    idx = np.argsort(distance)[0]
    label = best_centroids[idx][-1]
#     print(actual[i])
#     print('%s vs %s' % (label, actual[i]))
    if str(label) == str(actual[i]):
        count += 1
acc_kmode_knn = round(float(count)/len(actual)*100, 2)
print('The best accuracy for the task is %s' % acc_kmode_knn)

The best accuracy for the task is 99.04


In [51]:
# Decision Tree using Gain Ratio
import decisionTree_GR
train_data = pd.read_csv('train_processed.csv', dtype=str, usecols=range(1,10))
actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str)
test_data = pd.read_csv('test_processed.csv', usecols=range(1,9), dtype=str)
# print(train_data.head())
test_data = pd.concat([actual, test_data], axis=1)
# print(test_data.head())
actual = actual.values.flatten()
rows = train_data.values.tolist()
# print(rows[:2])
test_rows = test_data.values.tolist()
root = decisionTree_GR.buildtree(rows)
decisionTree_GR.prune(root, 0.49)
predictions = []
# print(actual)
for row in test_rows:
    predicted = decisionTree_GR.classify(row, root).most_common(1)[0][0]
    predictions.append(predicted)
# print(predictions)
acc_decisionTree_GR = round(decisionTree_GR.evaluate_accuracy(actual, predictions)*100, 2)
print('The best accuracy for C4.5 decision tree is %s' % acc_decisionTree_GR)

The best accuracy for C4.5 decision tree is 88.76


In [45]:
# Decision Tree using Information Gain
import decisionTree_IG
train_data = pd.read_csv('train_processed.csv', dtype=str, usecols=range(1,10))
actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str)
test_data = pd.read_csv('test_processed.csv', usecols=range(1,9), dtype=str)
# print(train_data.head())
test_data = pd.concat([actual, test_data], axis=1)
# print(test_data.head())
actual = actual.values.flatten()
rows = train_data.values.tolist()
# print(rows[:2])
test_rows = test_data.values.tolist()
root = decisionTree_IG.buildtree(rows)
decisionTree_IG.prune(root, 0.46)
predictions = []
# print(actual)
for row in test_rows:
    predicted = decisionTree_IG.classify(row, root).most_common(1)[0][0]
    predictions.append(predicted)
# print(predictions)
acc_decisionTree_IG = round(decisionTree_IG.evaluate_accuracy(actual, predictions)*100, 2)
print('The best accuracy for C4.5 decision tree is %s' % acc_decisionTree_IG)

The best accuracy for C4.5 decision tree is 84.69


In [48]:
# Naive Bayers 
import NaiveBayers
from collections import defaultdict
train_data = pd.read_csv('train_processed.csv', dtype=str, usecols=range(1,10))
actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str)
test_data = pd.read_csv('test_processed.csv', usecols=range(1,9), dtype=str)
test_data = pd.concat([actual, test_data], axis=1)
actual = actual.values.flatten()
rows = train_data.values.tolist()
test_rows = test_data.values.tolist()
classes = defaultdict(int)
# I create one more attribute that is unnecessary because I want to index to correspond to attributes
# attributes: list of dicts: key is 'classValue + attributeValue', value is its corresponding count
attributes = [defaultdict(int) for i in range(len(rows[0]))]
NaiveBayers.prepare(classes, attributes, rows)
acc_NaiveBayers = round(NaiveBayers.test(classes, attributes, test_rows, actual)*100, 2)
print('The accuracy of Naive Bayers is %s' % acc_NaiveBayers)       

The accuracy of Naive Bayers is 94.02


In [71]:
# Binary-perceptron
import perceptron
train_data = pd.read_csv('train_processed.csv', dtype=str, usecols=range(1,10))
actual = pd.read_csv('gender_submission.csv', usecols=[1], dtype=str)
test_data = pd.read_csv('test_processed.csv', usecols=range(1,9), dtype=str)
train_X = train_data.drop(['Survived'], axis=1)
train_Y = train_data['Survived']
test_X = test_data
test_Y = actual
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc_train_X = enc.fit_transform(train_X)
enc_test_X = enc.fit_transform(test_X)
enc_train_X = pd.DataFrame(enc_train_X.toarray())
enc_test_X = pd.DataFrame(enc_test_X.toarray())
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler(feature_range=(-1,1))
enc_train_X = min_max_scaler.fit_transform(enc_train_X)
enc_test_X = min_max_scaler.fit_transform(enc_test_X)
enc_train_X = pd.DataFrame(enc_train_X)
enc_test_X = pd.DataFrame(enc_test_X)
best_score =[]
lr_rates = [0.1,0.11,0.12,0.13,0.14]
# learning_rate =0.13 is the best
n_epochs = [15,20,22, 24, 25, 26, 28, 30]
# n_epoch is the best at 26
for n_epoch in n_epochs:
    best_score =[]
    for i in range(5):
        train_data = pd.concat([enc_train_X, train_Y], axis=1)
        test_X = enc_test_X
        actual = test_Y.values.astype('float')
        learning_rate = 0.13
        n_epoch = n_epoch
        score = perceptron.evaluate(train_data, test_X, actual, learning_rate, n_epoch)
    #     if score > best_score:
        best_score.append(score)
    acc_perceptron = np.mean(best_score)
    print('The accuracy for %d SGD is %f' % (n_epoch, acc_perceptron))

The accuracy for 15 SGD is 76.507177
The accuracy for 20 SGD is 75.502392
The accuracy for 22 SGD is 71.913876
The accuracy for 24 SGD is 70.095694
The accuracy for 25 SGD is 76.507177
The accuracy for 26 SGD is 76.937799
The accuracy for 28 SGD is 73.684211
The accuracy for 30 SGD is 71.770335


In [17]:
models = pd.DataFrame({
    'Model': ['kmodes with KNN', 'Multinomial Naive Bayers', 
              'Decision Tree Gain Ratio', 'Decision Tree Information Gain', 'Perceptron'],
    'Score': [acc_kmode_knn, acc_NaiveBayers, acc_decisionTree_GR, acc_decisionTree_IG, acc_perceptron]})
models.sort_values(by='Score', ascending=False)

,Model,Score
0,kmodes with KNN,98.090000
1,Multinomial Naive Bayers,94.020000
2,Decision Tree Gain Ratio,88.760000
3,Decision Tree Information Gain,84.690000
4,Perceptron,84.210526
